<a href="https://colab.research.google.com/github/cras-lab/OpenAPI/blob/main/SeoulMetroInfo_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필요데이터를 JSON으로 불러오기 위해, 필요한 모듈을 임포트 한다.

In [19]:
import urllib.request
import json

역이름과 날짜 시각을 주면, 해당 열차 정보를 반환하는 함수를 하나 정의한다.

In [27]:
def get_subway_info(api_key, station_name, date, time):
    # URL 인코딩을 위해 station_name을 인코딩합니다.
    station_name_encoded = urllib.parse.quote(station_name)
    url = f"http://swopenapi.seoul.go.kr/api/subway/{api_key}/json/realtimeStationArrival/0/5/{station_name_encoded}"
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read().decode('utf-8'))


            # 응답 데이터를 보기 쉽게 출력합니다.
        output = []
        output.append("Response Data:")
        output.append(pprint.pformat(data, indent=4, width=80, compact=False))

        if "errorMessage" in data:
            print("Error:", data["errorMessage"]["message"])
        else:
            arrivals = data['realtimeArrivalList']
            for arrival in arrivals:
                arrival_time = arrival['recptnDt'].split(' ')[1]
                if arrival['recptnDt'].startswith(date) and arrival_time.startswith(time):
                    print(f"역 이름: {arrival['statnNm']}")
                    print(f"열차 번호: {arrival['trainNo']}")
                    print(f"도착 예정 시간: {arrival['arvlMsg2']}")
                    print(f"현재 위치: {arrival['arvlMsg3']}")
                    print(f"운행 방향: {arrival['updnLine']}")
                    print("------------")
    return "\n".join(output)


서울열린데이터 광장에서 발급받은 API 키를 설정한다.

In [28]:
api_key = "42477864496f6b6133356455666f56"  # 이곳에는 각자 본인의 실제 발급받은 API 키를 입력하세요.

역이름과 날짜, 시각을 설정한다.

In [29]:
station_name = "서울"  # 검색할 지하철 역 이름
date = "2024-07-20"  # 검색할 날짜 (YYYY-MM-DD 형식)
time = "15:00"  # 검색할 시간 (HH:MM 형식)

서울열린데이터 광장의 지하철 정보에 직접 접속해서 값을 읽고, 출력한다.

In [ ]:
# 예제 호출
print( get_subway_info(api_key, station_name, date, time) )